In [1]:
# -*- coding: utf-8 -*-
"""
The script demonstrates a comparison of vehicle times from AVL data to the
scheduled arrival time at one stop for one example trip.
"""

import os
import pandas as pd

# these two modules are homemade
import gtfs
import arrivals
import time
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')


In [45]:

# get all the schedule data. (subset can be created later)
trips = gtfs.load_trips('gtfs/')
stops = gtfs.load_stops('gtfs/')
stop_times, tz_sched = gtfs.load_stop_times('gtfs/')
print 'Finished loading GTFS data.'

"""
# get the sample of parsed A	VL data.  Beware, takes a few minutes.
bustime = pd.read_csv('newdata_parsed.csv')#,parse_dates=dt_columns)
qstr = ('Trip == "MTA NYCT_MV_B6-Weekday-SDon-038500_M5_203" or '
    'Trip == "MTA NYCT_MV_B6-Weekday-SDon-036500_M5_202" or '
    'Trip == "MTA NYCT_MV_B6-Weekday-SDon-040000_M5_204"')
bustime = bustime.query(qstr)
bustime.drop_duplicates(['vehicleID','RecordedAtTime'],inplace=True)
bustime.set_index(['Line','Trip','TripDate','vehicleID','RecordedAtTime'],
                  inplace=True,drop=True,verify_integrity=True)

# for now, use a truncated data set.  just get data for one line (M5).
tripDateLookup = "2016-06-13"
lineLookup = "MTA NYCT_M5"
bustime_short = bustime.xs((lineLookup,tripDateLookup),level=(0,2))
del bustime # to free up memory
print 'Finished loading BusTime data and and slicing three trips of M5 line.'
"""

Finished loading GTFS data.


'\n# get the sample of parsed A\tVL data.  Beware, takes a few minutes.\nbustime = pd.read_csv(\'newdata_parsed.csv\')#,parse_dates=dt_columns)\nqstr = (\'Trip == "MTA NYCT_MV_B6-Weekday-SDon-038500_M5_203" or \'\n    \'Trip == "MTA NYCT_MV_B6-Weekday-SDon-036500_M5_202" or \'\n    \'Trip == "MTA NYCT_MV_B6-Weekday-SDon-040000_M5_204"\')\nbustime = bustime.query(qstr)\nbustime.drop_duplicates([\'vehicleID\',\'RecordedAtTime\'],inplace=True)\nbustime.set_index([\'Line\',\'Trip\',\'TripDate\',\'vehicleID\',\'RecordedAtTime\'],\n                  inplace=True,drop=True,verify_integrity=True)\n\n# for now, use a truncated data set.  just get data for one line (M5).\ntripDateLookup = "2016-06-13"\nlineLookup = "MTA NYCT_M5"\nbustime_short = bustime.xs((lineLookup,tripDateLookup),level=(0,2))\ndel bustime # to free up memory\nprint \'Finished loading BusTime data and and slicing three trips of M5 line.\'\n'

In [3]:
"""
# This function returns some elements of interest for each trip and stop
def trip_summary(avl_subset,t):    
    fail =  0
    trip_id = t[9:]
    stop_list = list(stop_times.loc[trip_id].index)
    # collect AVL data around each stop into a dict
    stop_pings = {}
    for stop_id in stop_list:
        try:
            stop_pings[stop_id] = arrivals.nearby_pings(stop_id,trip_id,stop_times,
                                                        stops,avl_subset)
        except:
            fail += 1
            continue    
    print str(time.time()) + ' - finished stop pings dict.'
    # make a summary table about the AVL data near each stop on a trip
    summary_columns = ['N','arrival_actual_estimated']
    summary_df = pd.DataFrame(columns=summary_columns,index=stop_pings.keys())
    for k, v in stop_pings.iteritems():
        # we want to know how many AVL pings were returned  nearby   
        newrow = {'N':len(v)}
        min_stamp = v['ResponseTimeStamp'].min()
        newrow['arrival_actual_estimated'] = min_stamp
        summary_df.loc[k] = newrow
    print str(time.time()) + ' - finished ' + t[9:]
    return summary_df

# now collect a dataframe with the summary data about all trips in the subset
day_summary = pd.DataFrame()
for t in bustime_short.index.get_level_values(0).unique():
    try:
        avl_subset = bustime_short.loc[t]
        summary_df = trip_summary(avl_subset,t)
        summary_df['trip_id'] = t[9:]
        summary_df['TripDate'] = tripDateLookup
        summary_df.reset_index(inplace=True)
        summary_df.rename(columns={'index':'stop_id'},inplace=True)
        day_summary = day_summary.append(summary_df)
    except:
        print t
day_summary.set_index(['trip_id','stop_id'],inplace=True)
day_summary = day_summary.join(stop_times['arrival_time'])

print day_summary.shape
"""


"\n# This function returns some elements of interest for each trip and stop\ndef trip_summary(avl_subset,t):    \n    fail =  0\n    trip_id = t[9:]\n    stop_list = list(stop_times.loc[trip_id].index)\n    # collect AVL data around each stop into a dict\n    stop_pings = {}\n    for stop_id in stop_list:\n        try:\n            stop_pings[stop_id] = arrivals.nearby_pings(stop_id,trip_id,stop_times,\n                                                        stops,avl_subset)\n        except:\n            fail += 1\n            continue    \n    print str(time.time()) + ' - finished stop pings dict.'\n    # make a summary table about the AVL data near each stop on a trip\n    summary_columns = ['N','arrival_actual_estimated']\n    summary_df = pd.DataFrame(columns=summary_columns,index=stop_pings.keys())\n    for k, v in stop_pings.iteritems():\n        # we want to know how many AVL pings were returned  nearby   \n        newrow = {'N':len(v)}\n        min_stamp = v['ResponseTimeStamp

In [46]:
# day_summary.to_csv('day_summary.csv')

data = pd.read_csv('longjsons_parsed.csv')

qstr = ('Trip == "MTA NYCT_FB_B6-Weekday-SDon-043400_B41_16"')
bustime = data.query(qstr)
bustime.drop_duplicates(['vehicleID','RecordedAtTime'],inplace=True)
bustime.set_index(['Line','Trip','TripDate','vehicleID','RecordedAtTime'],
                  inplace=True,drop=True,verify_integrity=True)
# for now, use a truncated data set.  just get data for one line (M5).
tripDateLookup = "2016-05-25"
lineLookup = "MTA NYCT_B41"
avl_long = bustime.xs((lineLookup,tripDateLookup),level=(0,2))
del bustime,data # to free up memory



In [13]:
from scipy import spatial

In [81]:
avl_data = avl_long
radius = 0.01
trip_id = "FB_B6-Weekday-SDon-043400_B41_16"
dist_route = 2600
#lat, lon = 40.6763783,-73.983488

trip_stop_times = stop_times.loc[trip_id]
lookup = 'MTA NYCT_' + trip_id # lookup string used to query AVL data
avl_subset = avl_data.query('Trip == @lookup')
#print avl_subset
points = zip(avl_subset.CallDistAlongRoute.ravel())
tree = spatial.KDTree(points) # implement KDTree class
lookup_point = [dist_route]
nearby_points = tree.query_ball_point(lookup_point, radius)
#print nearby_points
locations = avl_subset.iloc[nearby_points][['CallDistAlongRoute']]
times = avl_subset.iloc[nearby_points].index.get_level_values('RecordedAtTime')
df = pd.DataFrame(avl_subset.iloc[nearby_points][['CallDistAlongRoute']])
df['RecordedAtTime'] = times
df = df.set_index(['RecordedAtTime'])
#resampled = df.resample('S').interpolate()
return df#resampled


time_at_location(dist_route,trip_id,avl_data,stop_times,radius)    

ValueError: Troublesome data array: [  2387.92   2598.69   2598.69   2598.69   2981.87   2981.87   2981.87
   3197.86   3197.86   3406.24   3695.94   3695.94   3695.94   3695.94
   4539.73   4539.73   4539.73   4539.73   4908.69   4908.69   5120.66
   5120.66   5120.66   5436.26   5436.26   5436.26   5436.26   5766.06
   5766.06   5766.06   5766.06   6065.31   6065.31   6393.44   6393.44
   6393.44       nan       nan   6677.85   6978.25   6978.25   6978.25
   6978.25   7252.59   7252.59   7252.59   7498.74   7498.74   7785.86
   7785.86   7785.86   7785.86   8146.94   8146.94   8146.94   8146.94
   8360.25   8360.25   8360.25   8675.57   8675.57   8675.57   8675.57
   8675.57   8976.29   9298.28   9298.28   9689.41   9689.41  10075.25
  10075.25  10075.25  10075.25  10248.78  10248.78  10248.78  10562.79
  10562.79  10562.79  10562.79       nan  10862.42  10862.42  11111.04
  11111.04  11111.04  11111.04  11266.13  11429.64  11598.19  11598.19
  11759.84  11759.84  11869.77  12057.53 

In [76]:
avl_data.head()

Latitude  \
Trip                                      vehicleID     RecordedAtTime                             
MTA NYCT_FB_B6-Weekday-SDon-043400_B41_16 MTA NYCT_4587 2016-05-25T07:26:40.000-04:00  40.682170   
                                                        2016-05-25T07:27:11.000-04:00  40.681098   
                                                        2016-05-25T07:27:42.000-04:00  40.681126   
                                                        2016-05-25T07:28:14.000-04:00  40.680564   
                                                        2016-05-25T07:28:45.000-04:00  40.678978   

                                                                                       Longitude  \
Trip                                      vehicleID     RecordedAtTime                             
MTA NYCT_FB_B6-Weekday-SDon-043400_B41_16 MTA NYCT_4587 2016-05-25T07:26:40.000-04:00 -73.976072   
                                                        2016-05-25T07:27:11.000-04:00 -73.975304   
                                                        2016-05-25T07:27:42.000-04:00 -73.975324   
                                                        2016-05-25T07:28:14.000-04:00 -73.974913   
                                                        2016-05-25T07:28:45.000-04:00 -73.973757   

                                                                                       TripPattern  \
Trip                                      vehicleID     RecordedAtTime                               
MTA NYCT_FB_B6-Weekday-SDon-043400_B41_16 MTA NYCT_4587 2016-05-25T07:26:40.000-04:00  MTA_B410332   
                                                        2016-05-25T07:27:11.000-04:00  MTA_B410332   
                                                        2016-05-25T07:27:42.000-04:00  MTA_B410332   
                                                        2016-05-25T07:28:14.000-04:00  MTA_B410332   
                                                        2016-05-25T07:28:45.000-04:00  MTA_B410332   

                                                                                      MonitoredCallRef  \
Trip                                      vehicleID     RecordedAtTime                                   
MTA NYCT_FB_B6-Weekday-SDon-043400_B41_16 MTA NYCT_4587 2016-05-25T07:26:40.000-04:00       MTA_901500   
                                                        2016-05-25T07:27:11.000-04:00       MTA_303297   
                                                        2016-05-25T07:27:42.000-04:00       MTA_303297   
                                                        2016-05-25T07:28:14.000-04:00       MTA_303297   
                                                        2016-05-25T07:28:45.000-04:00       MTA_303300   

                                                                                                      EstCallArrival  \
Trip                                      vehicleID     RecordedAtTime                                                 
MTA NYCT_FB_B6-Weekday-SDon-043400_B41_16 MTA NYCT_4587 2016-05-25T07:26:40.000-04:00  2016-05-25T07:27:13.126-04:00   
                                                        2016-05-25T07:27:11.000-04:00  2016-05-25T07:27:43.111-04:00   
                                                        2016-05-25T07:27:42.000-04:00  2016-05-25T07:28:13.158-04:00   
                                                        2016-05-25T07:28:14.000-04:00  2016-05-25T07:28:43.133-04:00   
                                                        2016-05-25T07:28:45.000-04:00  2016-05-25T07:29:38.356-04:00   

                                                                                       DistFromCall  \
Trip                                      vehicleID     RecordedAtTime                                
MTA NYCT_FB_B6-Weekday-SDon-043400_B41_16 MTA NYCT_4587 2016-05-25T07:26:40.000-04:00          0.00   
                                                        2016-05-25T07:27:11.00